In [1]:
import monai

monai.config.print_config()

MONAI version: 0.4.0+9.g36b8302
Numpy version: 1.19.2
Pytorch version: 1.7.0
MONAI flags: HAS_EXT = True, USE_COMPILED = False
MONAI rev id: 36b8302f206104c3743197c5d624badb2806f239

Optional dependencies:
Pytorch Ignite version: 0.4.2
Nibabel version: 3.1.1
scikit-image version: 0.17.2
Pillow version: 8.0.1
Tensorboard version: 2.3.0
gdown version: 3.12.2
TorchVision version: 0.8.1
ITK version: 5.2.0
tqdm version: 4.51.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.7.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
# Include numpy for pi
import numpy as np

# Include monai applications: which are collections of data already allocated to training, testing, and/or validation sets
from monai.apps import DecathlonDataset

# Include monai's standard data loader
from monai.data import DataLoader

# Include transforms that are used to normalize and augment data as part of the training workflow
from monai.transforms import \
    LoadImageD, LoadNiftiD, AsChannelFirstD, AsChannelLastD, AddChannelD, ScaleIntensityD, ToTensorD, Compose, \
    AsDiscreteD, SpacingD, OrientationD, ResizeD, RandAffineD, ThresholdIntensityD

# Enable deterministic training for python, numpy and pytorh
monai.utils.set_determinism(seed=0, additional_settings=None)

# Specify where the collections of downloaded data should be stored on the local filesystem
root_dir = "./"

# The training transform is a composition of transforms that manipulate the images and labels (segmentation truth) as a
#    data processing pipeline.
train_transform = Compose([                   # These are the transforms to be associated with the training data
    LoadImageD(keys=["image","label"]),          # The Decathlan data has images and labels, and both should be loaded
    AsChannelFirstD("image"),                    # The first dimenion of the 4D image specifies the data channel: e.g., T1 or T2
    AddChannelD("label"),                        # Then we associate the ideal segmentations with each image
    SpacingD(keys=["image","label"], pixdim=(1., 1., 1.), mode=('bilinear', 'nearest')),   # images and labels are resmpled to
                                                                                           #   isotropic, 1x1x1mm voxels
    OrientationD(keys=["image","label"], axcodes='RAS'),                        # Images and labels are oriented into RAS space
    ScaleIntensityD(keys=["image"]),                                            # Each image's intensities are normalized 0 to 1
    ThresholdIntensityD(keys=["label"], threshold=0.5, above=False, cval=1),    # We reduced the labels from 3 classes to 2,
                                                                                #   by thresholding keeping background as 0, and
                                                                                #   reassigning labels 1 and 2 as 1.
    RandAffineD(keys=["image","label"], spatial_size=(-1, -1, -1),   # When an image is requested for training, with probability
                rotate_range=(0, 0, np.pi/8),                        #   of 1, it will be rotated in the z-axis, scaled in every
                scale_range=(0.1, 0.1, 0.1),                         #   axis, and resamples using bilinary interpolation for 
                mode=('bilinear', 'nearest'),                        #   images and nearest neighbor interpolation for labels
                prob=1.0),
    ToTensorD(keys=["image","label"]),                               # Images and labels are then exported as tensors
])

# The  training data is defined as Task 05 from the Decathlon challenge.  The DecathlonDataset knows the data's location on the
#   web and its MD5 checksum.  We specify where it should be stored on the local filesystem. Furthermore, we specify that the 
#   data should be processed using the training transforms defined above, before being provided to a MONAI network. 
#   Via Datasets, data preparation is managed separately from the details of network training.  This separation enables the 
#   sharing of standard training data and the clear (reproducible) definition of data pre-processing and augmentation steps.
#   Note that a dataset is merely a definition.  The actual data downloading and processing occurs with the data loader 
#   command (next).   Additionally, datasets optionally offer caching of intermediate results, and other optimization 
#   specifications.
train_dataset = DecathlonDataset(root_dir=root_dir, task="Task05_Prostate", section="training", \
                                 transform=train_transform, download=True)

# The data is the downloaded, MD5 verified, processed, and cached as specified by the dataset
train_data_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0)

  0%|                                                   | 0/26 [00:00<?, ?it/s]

Verified 'Task05_Prostate.tar', md5: 35138f08b1efaef89d7424d2bcc928db.
file ./Task05_Prostate.tar exists, skip downloading.
extracted file ./Task05_Prostate exists, skip extracting.


Modifying image pixdim from [0.625   0.625   3.60001 1.     ] to [  0.625        0.625        3.60000992 151.21210251]
100%|██████████████████████████████████████████| 26/26 [00:15<00:00,  1.67it/s]


In [3]:
# ITKWidgets provide interactive 2D and 3D visualizations of ITK images, Pytorch/MONAI tensors, and multiple other image
#   surface, mesh, and point-cloud formats.   It is vital to verify inputs, intermediate results, and outputs using
#   a high-quality medical image aware viewer such as ITKWidgets with conducting deep learning research - otherwise days of
#   training can be wasted or new insights missed...
from itkwidgets import view

# The data loader provides access to training data.  Transforms are applied to the data as defined.  Intermediate results 
#   (e.g., prior to application of random transforms) are cached when possible to speed the generation of additional data.
data_dict = next(iter(train_data_loader))

# The itkwidget view command accepts multiple arguments to define 3D and 2D visualization parameters.  Here the first training
#   image is shown with its segmentation label shown as an overlay.   The user can swith between 2D and 3D views and specify
#   transfer functions, window and level settings, overlap opacity, crop regions, landmark locations, and much more.
view(image=data_dict["image"][0,0,:,:,:], label_image=data_dict["label"][0,0,:,:,:])

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…

In [4]:
import torch

# For this demo, we will illustrate how UNet (and any of the 20+ networks available in MONAI) can be easily specified
#   using a few parameters.
from monai.networks.nets import UNet

# This network will use Batch normalization, the Dice loss function commonly used in medical image segmentation performance
#   assessment, and a SlidingWindowInferer that partitions arbitrarily large images into sections that are streamed
#   individually to the GPU, to reduce GPU memory requirements.
from monai.networks.layers import Norm
from monai.losses import DiceLoss
from monai.inferers import SlidingWindowInferer

# A local GPU is assumed.
device = torch.device('cuda:0')

# The specification of a UNet architecture has been reduced to its essential parameters in a compact yet intuitive format
net = UNet(dimensions=3, in_channels=2, out_channels=2, channels=(16, 32, 64, 128, 256),
           strides=(2, 2, 2, 2), num_res_units=2, norm=Norm.BATCH).to(device)

# The background (0 value labels) are ignored in dice computation to focus performance assessment on the class of interest
loss = DiceLoss(to_onehot_y=True, softmax=True, include_background=False)

# Adam optimization from PyTorch (and other aspects of Pytorch) can be easily combined with MONAI workflows
opt = torch.optim.Adam(net.parameters(), 1e-4)

# The sliding windows overlap by 50%, and results from multiple windows are combined based on a gaussian weighting of their
#    distance from the center of each window.
inferer = SlidingWindowInferer((32, 128, 128), sw_batch_size=1, overlap=0.5, mode='gaussian')

In [5]:
# Engines provide training and inference workflows
from monai.engines import SupervisedTrainer

# Handlers are called when events occur during training and inference (e.g., between epochs)
from monai.handlers import StatsHandler, MeanDice

# Outstanding documentation on available transforms is online at https://docs.monai.io/en/latest/
from monai.transforms import AsDiscreteD

# Setup information-level logging
import sys
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# After passing data through the network to generate a prediction,  The unet's 2 outputs are converted to class labels via
#   argmax), the prediction and labels are compared by converting both to a 2-class, one-hot encoding that dice expects.
train_post_transform = Compose([
    AsDiscreteD(keys=["pred", "label"], argmax=(True, False), to_onehot=True, n_classes=2)
])

# After each epoch, report the loss function's value
train_handler = [
    StatsHandler(tag_name="train_loss", output_transform=lambda x: x["loss"]),
]

# Pass the network and optimization details to the training engine, and run for 20 epochs.
trainer = SupervisedTrainer(
    device=device,
    max_epochs=20,
    train_data_loader=train_data_loader,
    network=net,
    optimizer=opt,
    loss_function=loss,
    inferer=inferer,
    post_transform=train_post_transform,
    train_handlers=train_handler,
    key_train_metric={"train_meandice": MeanDice(include_background=False, output_transform=lambda x: (x["pred"], x["label"]))},
)

trainer.run()

INFO:ignite.engine.engine.SupervisedTrainer:Engine run resuming from iteration 0, epoch 0 until 20 epochs
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 1/26 -- train_loss: 0.9482 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 2/26 -- train_loss: 0.9663 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 3/26 -- train_loss: 0.9762 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 4/26 -- train_loss: 0.9622 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 5/26 -- train_loss: 0.9616 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 6/26 -- train_loss: 0.9120 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 7/26 -- train_loss: 0.9308 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 8/26 -- train_loss: 0.8888 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 9/26 -- train_loss: 0.9272 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 1/20, Iter: 10/26 -- tra

INFO:ignite.engine.engine.SupervisedTrainer:Key metric: train_meandice best value: 0.3802880490055451 at epoch: 3
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[3] Complete. Time taken: 00:00:57
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 1/26 -- train_loss: 0.8583 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 2/26 -- train_loss: 0.7269 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 3/26 -- train_loss: 0.8590 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 4/26 -- train_loss: 0.7752 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 5/26 -- train_loss: 0.8576 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 6/26 -- train_loss: 0.7154 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 7/26 -- train_loss: 0.8967 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 8/26 -- train_loss: 0.8653 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 4/20, Iter: 9/26 -- tra

INFO:ignite.engine.engine.SupervisedTrainer:Epoch[6] Metrics -- train_meandice: 0.6510 
INFO:ignite.engine.engine.SupervisedTrainer:Key metric: train_meandice best value: 0.6509596888835614 at epoch: 6
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[6] Complete. Time taken: 00:00:58
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 1/26 -- train_loss: 0.4949 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 2/26 -- train_loss: 0.6791 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 3/26 -- train_loss: 0.7093 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 4/26 -- train_loss: 0.4604 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 5/26 -- train_loss: 0.4876 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 6/26 -- train_loss: 0.4949 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 7/26 -- train_loss: 0.4165 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 7/20, Iter: 8/26 -- train_

INFO:ignite.engine.engine.SupervisedTrainer:Got new best metric of train_meandice: 0.7861886895619906
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[9] Metrics -- train_meandice: 0.7862 
INFO:ignite.engine.engine.SupervisedTrainer:Key metric: train_meandice best value: 0.7861886895619906 at epoch: 9
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[9] Complete. Time taken: 00:00:56
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 10/20, Iter: 1/26 -- train_loss: 0.4890 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 10/20, Iter: 2/26 -- train_loss: 0.3262 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 10/20, Iter: 3/26 -- train_loss: 0.4088 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 10/20, Iter: 4/26 -- train_loss: 0.3762 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 10/20, Iter: 5/26 -- train_loss: 0.3467 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 10/20, Iter: 6/26 -- train_loss: 0.4308 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 10/20, It

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 12/20, Iter: 26/26 -- train_loss: 0.2291 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[12] Metrics -- train_meandice: 0.7726 
INFO:ignite.engine.engine.SupervisedTrainer:Key metric: train_meandice best value: 0.8060809465555044 at epoch: 10
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[12] Complete. Time taken: 00:00:56
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 13/20, Iter: 1/26 -- train_loss: 0.4522 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 13/20, Iter: 2/26 -- train_loss: 0.5094 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 13/20, Iter: 3/26 -- train_loss: 0.1777 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 13/20, Iter: 4/26 -- train_loss: 0.2953 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 13/20, Iter: 5/26 -- train_loss: 0.2627 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 13/20, Iter: 6/26 -- train_loss: 0.2506 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 13/20, Iter: 7/

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 15/20, Iter: 26/26 -- train_loss: 0.3288 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[15] Metrics -- train_meandice: 0.8005 
INFO:ignite.engine.engine.SupervisedTrainer:Key metric: train_meandice best value: 0.8147934468892905 at epoch: 14
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[15] Complete. Time taken: 00:00:58
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 16/20, Iter: 1/26 -- train_loss: 0.1834 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 16/20, Iter: 2/26 -- train_loss: 0.1749 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 16/20, Iter: 3/26 -- train_loss: 0.2695 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 16/20, Iter: 4/26 -- train_loss: 0.1903 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 16/20, Iter: 5/26 -- train_loss: 0.2709 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 16/20, Iter: 6/26 -- train_loss: 0.3150 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 16/20, Iter: 7/

INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 18/20, Iter: 25/26 -- train_loss: 0.1616 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 18/20, Iter: 26/26 -- train_loss: 0.2669 
INFO:ignite.engine.engine.SupervisedTrainer:Got new best metric of train_meandice: 0.8626898229122162
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[18] Metrics -- train_meandice: 0.8627 
INFO:ignite.engine.engine.SupervisedTrainer:Key metric: train_meandice best value: 0.8626898229122162 at epoch: 18
INFO:ignite.engine.engine.SupervisedTrainer:Epoch[18] Complete. Time taken: 00:00:56
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 19/20, Iter: 1/26 -- train_loss: 0.0974 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 19/20, Iter: 2/26 -- train_loss: 0.1801 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 19/20, Iter: 3/26 -- train_loss: 0.1437 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 19/20, Iter: 4/26 -- train_loss: 0.2251 
INFO:ignite.engine.engine.SupervisedTrainer:Epoch: 19/2

In [6]:
from monai.transforms import \
    ActivationsD, KeepLargestConnectedComponentD
from monai.handlers import SegmentationSaver
from monai.engines import SupervisedEvaluator
from monai.inferers import SlidingWindowInferer
from ignite.metrics import Accuracy

# For validation, the Decathlon dataset only provides images (no labels), and we do not want to conduct random rotations - we
#   should only process the raw data, albeit resampled to isotropic voxels to match the training data's resolution
val_transforms = Compose([
    LoadImageD(keys=["image"]),
    AsChannelFirstD("image"),
    SpacingD(keys=["image"], pixdim=(1., 1., 1.), mode=('bilinear')),
    OrientationD(keys=["image"], axcodes='RAS'),
    ScaleIntensityD(keys="image"),
    ToTensorD(keys=["image"]),
])

# After a validation image is passed through the unet, we use argmax to convert from one-hot encoding to a single label image.
#   Additionally, we use the "KeepLargestConnectedComponent" filter to eliminate pixel label noise
val_post_transform = Compose([
    AsDiscreteD(keys=["pred"], argmax=True),
    KeepLargestConnectedComponentD(keys=["pred"], applied_labels=[1], connectivity=1, independent=False)
])

# Here we defined the validation dataset, from the Decathlon challenge, with minimal transforms applied.
val_dataset = DecathlonDataset(root_dir="./", task="Task05_Prostate", section="test", transform=val_transforms, download=True)

# The validation is downloaded, if it hasn't been already, as specified by the validation dataset.
val_data_loader = DataLoader(val_dataset, batch_size=1, num_workers=0)

# The segmentation results (predictions) from the unet will be saved to the local filesystem
output_dir = "./runs/"

# A MONAI handler is called after each validation image / batch is processed, to save the results.
val_handler = [
    StatsHandler(output_transform=lambda x: None),
    SegmentationSaver(
        output_dir=output_dir,
        batch_transform=lambda batch: batch["image_meta_dict"],
        output_transform=lambda output: output["pred"],
    ),
]

# The validation data loader, the already trained network, sliding window inferer, post transform, and handler are passed
#   to the inference engine.
evaluator = SupervisedEvaluator(
    device=device,
    val_data_loader=val_data_loader,
    network=net,
    inferer=inferer,
    post_transform=val_post_transform,
    val_handlers=val_handler,
)

evaluator.run()

  0%|                                                   | 0/16 [00:00<?, ?it/s]

Verified 'Task05_Prostate.tar', md5: 35138f08b1efaef89d7424d2bcc928db.
file ./Task05_Prostate.tar exists, skip downloading.
extracted file ./Task05_Prostate exists, skip extracting.


Modifying image pixdim from [0.6       0.5999997 3.999998  1.       ] to [  0.60000002   0.59999975   3.99999799 146.48690756]
100%|██████████████████████████████████████████| 16/16 [00:06<00:00,  2.64it/s]

INFO:ignite.engine.engine.SupervisedEvaluator:Engine run resuming from iteration 0, epoch 0 until 1 epochs


INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved all the model outputs into files.
INFO:ignite.engine.engine.SupervisedEvaluator:saved al

In [20]:
# Once the validation data has been processed and the predicted segmentations have been saved to disk, we can use
#   ITK and ITKWidgets to read and interactively explore the validation images with predicted segmentations overlaid...

import glob
import os

import itk

from itkwidgets import view

root_dir = "./"
output_dir = "./runs/"

imList=sorted(glob.glob(os.path.join(root_dir, "Task05_Prostate/imagesTs", "*.nii.gz")))
segList=sorted(glob.glob(os.path.join(output_dir, "*", "*.nii.gz")))

im = itk.imread(imList[0])
seg = itk.imread(segList[0])

view(image=im[0,:,:,:], label_image=seg)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…